In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import random
import os
import tensorflow
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
tensorflow.set_random_seed(SEED)
os.environ['PYTHONHASHSEED']=str(SEED)

#### The Data

In [4]:
#df = pd.read_csv('data/anemia_synth_dataset_hb_some_nans.csv') #my real dataset i think
#df = pd.read_csv('data/anemia_synth_dataset_hb.csv')
df = pd.read_csv('data/noisy_data_uniform_all_30_08_22.csv')
df = df.fillna(0)
classes = list(df.label.unique())
nums = [i for i in range(len(classes))]
class_dict = dict(zip(classes, nums))
class_dict

{'Hemolytic anemia': 0,
 'Anemia of chronic disease': 1,
 'No anemia': 2,
 'Aplastic anemia': 3,
 'Vitamin B12/Folate deficiency anemia': 4,
 'Iron deficiency anemia': 5}

In [5]:
df['label'] = df['label'].replace(class_dict)
X = df.iloc[:, 0:-1]
y = df.iloc[:, -1]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=SEED)
X_train, y_train = np.array(X_train), np.array(y_train)
X_test, y_test = np.array(X_test), np.array(y_test)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((42000, 6), (18000, 6), (42000,), (18000,))

In [6]:
action_list = list(class_dict.keys()) + [col  for col in df.columns if col!='label']
len(action_list)

12

In [7]:
action_list

['Hemolytic anemia',
 'Anemia of chronic disease',
 'No anemia',
 'Aplastic anemia',
 'Vitamin B12/Folate deficiency anemia',
 'Iron deficiency anemia',
 'hemoglobin',
 'ferritin',
 'ret_count',
 'segmented_neutrophils',
 'tibc',
 'mcv']

In [8]:
X_train[:2], X_test[:2]

(array([[  1.87583201, 343.18430633,   4.98410354,   0.        ,
           0.        ,  87.18608967],
        [  1.57074342,   0.        ,   3.37668014,   2.06695952,
           0.        ,  94.71994026]]),
 array([[  6.24850792,   0.        ,   0.5241004 ,   0.        ,
         373.64490188,  98.79429654],
        [  7.62067707, 127.79573525,   0.        ,   1.74847797,
          82.66968799,  77.72260827]]))

#### The Environment

In [9]:
from envs import SyntheticComplexHbEnv

#### The Agent

In [10]:
from stable_baselines.common.env_checker import check_env
from stable_baselines.common.policies import MlpPolicy
from stable_baselines.common.vec_env import DummyVecEnv
from stable_baselines import PPO2
from stable_baselines import DQN
from stable_baselines import bench, logger

C:\Users\User\Anaconda3\envs\tf_v1_env\lib\site-packages\stable_baselines\__init__.py:33: UserWarning: stable-baselines is in maintenance mode, please use [Stable-Baselines3 (SB3)](https://github.com/DLR-RM/stable-baselines3) for an up-to-date version. You can find a [migration guide](https://stable-baselines3.readthedocs.io/en/master/guide/migration.html) in SB3 documentation.
  "stable-baselines is in maintenance mode, please use [Stable-Baselines3 (SB3)](https://github.com/DLR-RM/stable-baselines3) for an up-to-date version. You can find a [migration guide](https://stable-baselines3.readthedocs.io/en/master/guide/migration.html) in SB3 documentation."


In [11]:
def stable_dqn():
    training_env = SyntheticComplexHbEnv(X_train, y_train)
    env = bench.Monitor(training_env, logger.get_dir())
    model = DQN('MlpPolicy', training_env, verbose=1, seed=SEED, n_cpu_tf_sess=1)
    model.learn(total_timesteps=int(4.1e6), log_interval=10000)
    #model.learn(total_timesteps=int(1.2e5), log_interval=10000)
    #model.save('models/synthetic_stable_dqn_1.8.pkl')
    model.save('models/noisy/dqn_uniform_all_41e6.pkl')
    env.close()
    return model

dqn_model = stable_dqn()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Use tf.cast instead.
--------------------------------------
| % time spent exploring  | 94       |
| episodes                | 10000    |
| mean 100 episode reward | 0.1      |
| steps                   | 20514    |
| success rate            | 0.18     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 89       |
| episodes                | 20000    |
| mean 100 episode reward | 0.2      |
| steps                   | 42112    |
| success rate            | 0.13     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 83       |
| episodes                | 30000    |
| mean 100 episode reward | 0.1      |
| steps                   | 64706    |
| success rate            | 0.12     |
--------------------------------------


--------------------------------------
| % time spent exploring  | 2        |
| episodes                | 280000   |
| mean 100 episode reward | 2.8      |
| steps                   | 1000073  |
| success rate            | 0.67     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 2        |
| episodes                | 290000   |
| mean 100 episode reward | 2.7      |
| steps                   | 1041452  |
| success rate            | 0.69     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 2        |
| episodes                | 300000   |
| mean 100 episode reward | 3        |
| steps                   | 1083477  |
| success rate            | 0.73     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 2        |
| episodes                | 310000   |
| mean 100 episode reward | 2.6      |
| steps                  

--------------------------------------
| % time spent exploring  | 2        |
| episodes                | 590000   |
| mean 100 episode reward | 2.6      |
| steps                   | 2276465  |
| success rate            | 0.69     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 2        |
| episodes                | 600000   |
| mean 100 episode reward | 3.2      |
| steps                   | 2317425  |
| success rate            | 0.76     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 2        |
| episodes                | 610000   |
| mean 100 episode reward | 2.9      |
| steps                   | 2359085  |
| success rate            | 0.77     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 2        |
| episodes                | 620000   |
| mean 100 episode reward | 3.3      |
| steps                  

--------------------------------------
| % time spent exploring  | 2        |
| episodes                | 900000   |
| mean 100 episode reward | 3.2      |
| steps                   | 3578957  |
| success rate            | 0.79     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 2        |
| episodes                | 910000   |
| mean 100 episode reward | 2.7      |
| steps                   | 3620739  |
| success rate            | 0.71     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 2        |
| episodes                | 920000   |
| mean 100 episode reward | 2.6      |
| steps                   | 3663006  |
| success rate            | 0.7      |
--------------------------------------
--------------------------------------
| % time spent exploring  | 2        |
| episodes                | 930000   |
| mean 100 episode reward | 2.8      |
| steps                  

#### Performance Evaluation

In [12]:
#dqn_model = DQN.load('models/noisy/dqn_uniform_all_4e6.pkl')

In [13]:
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, auc, roc_curve

In [14]:
def multiclass(actual_class, pred_class, average = 'macro'):

    unique_class = set(actual_class)
    roc_auc_dict = {}
    for per_class in unique_class:
        other_class = [x for x in unique_class if x != per_class]
        new_actual_class = [0 if x in other_class else 1 for x in actual_class]
        new_pred_class = [0 if x in other_class else 1 for x in pred_class]
        roc_auc = roc_auc_score(new_actual_class, new_pred_class, average = average)
        roc_auc_dict[per_class] = roc_auc
    avg = sum(roc_auc_dict.values()) / len(roc_auc_dict)
    return avg

In [15]:
def test(ytest, ypred):
    acc = accuracy_score(ytest, ypred)
    f1 = f1_score(ytest, ypred, average ='macro', labels=np.unique(ytest))
    try:
        roc_auc = multiclass(ytest, ypred)
    except:
        roc_auc = None
    return acc, f1, roc_auc

In [16]:
def get_avg_length_reward(df):
    length = np.mean(df.episode_length)
    reward = np.mean(df.reward)
    return length, reward

In [17]:
def synthetic_dqn_eval(dqn_model):
    attempts, correct = 0,0
    test_df = pd.DataFrame()

    env = SyntheticComplexHbEnv(X_test, y_test, random=False)
    #env = SyntheticComplexHbEnv(X_train, y_train, random=False)
    count=0

    try:
        while True:
            count+=1
            if count%5000==0:
                print(f'Count: {count}')
            obs, done = env.reset(), False
            while not done:
                action, _states = dqn_model.predict(obs, deterministic=True)
                obs, rew, done,info = env.step(action)
                #if (done==True) & (np.isfinite(info['y_pred'])):
                if done == True:
                    test_df = test_df.append(info, ignore_index=True)
                #print('....................TEST DF ....................')
                #if len(test_df) != 0:
                #    print(test_df.head())

    except StopIteration:
        print('Testing done.....')
    return test_df

test_df = synthetic_dqn_eval(dqn_model)

Count: 5000
Count: 10000
Count: 15000
Testing done.....


In [18]:
len(X_test), len(test_df)

(18000, 18000)

In [19]:
y_pred_df = test_df[test_df['y_pred'].notna()]
success_df = y_pred_df[y_pred_df['y_pred']== y_pred_df['y_actual']]
len(success_df)

14707

In [20]:
# test_df.to_csv('test_dfs/noisy/final_test_df_4e6.csv', index=False)
# y_pred_df.to_csv('test_dfs/noisy/final_y_pred_df_4e6.csv', index=False)
# success_df.to_csv('test_dfs/noisy/final_success_df_4e6.csv', index=False)

In [21]:
y_pred_df.y_pred.unique()

array([0., 1., 3., 2., 4.])

In [22]:
y_pred_df.y_pred.value_counts()

0.0    6768
3.0    3162
2.0    3144
1.0    2430
4.0     476
Name: y_pred, dtype: int64

In [23]:
y_pred_df.y_actual.value_counts()

0.0    6621
3.0    3161
2.0    2971
1.0    2331
4.0     788
5.0     108
Name: y_actual, dtype: int64

In [24]:
success_rate = len(success_df)/len(test_df)*100
success_rate

81.70555555555555

In [25]:
#avg length and return 
avg_length, avg_return = get_avg_length_reward(test_df)
avg_length, avg_return

(4.359111111111111, 3.1145555555555555)

In [26]:
acc, f1, roc_auc = test(y_pred_df['y_actual'], y_pred_df['y_pred'])
acc, f1, roc_auc

(0.9203379224030037, 0.7010800996693796, 0.8395336148701881)

#### Confusion Matrix and Classification Report

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

In [ ]:
def plot_confusion_matrix(cm, save=False, filename=False):
    cm_df = pd.DataFrame(cm, index = [0, 1, 2, 3, 4, 5], columns = [0, 1, 2, 3, 4, 5])
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm_df, annot=True)
    plt.title('Confusion Matrix')
    plt.ylabel('Actual Anemia')
    plt.xlabel('Predicted Anemia')
    plt.tight_layout()
    if save:
        plt.savefig(filename)
    plt.show()
    plt.close()

In [ ]:
plot_confusion_matrix(confusion_matrix(y_pred_df['y_actual'], y_pred_df['y_pred']))

In [ ]:
def cm2inch(*tupl):
    inch = 2.54
    if type(tupl[0]) == tuple:
        return tuple(i/inch for i in tupl[0])
    else:
        return tuple(i/inch for i in tupl)

In [ ]:
def show_values(pc, fmt="%.2f", **kw):    
    pc.update_scalarmappable()
    ax = pc.axes
    for p, color, value in zip(pc.get_paths(), pc.get_facecolors(), pc.get_array()):
        x, y = p.vertices[:-2, :].mean(0)
        if np.all(color[:3] > 0.5):
            color = (0.0, 0.0, 0.0)
        else:
            color = (1.0, 1.0, 1.0)
        ax.text(x, y, fmt % value, ha="center", va="center", color=color, **kw)

In [ ]:
def heatmap(AUC, title, xlabel, ylabel, xticklabels, yticklabels, figure_width=40, figure_height=20, correct_orientation=False, cmap='RdBu'):
    fig, ax = plt.subplots()    
    c = ax.pcolor(AUC, edgecolors='k', linestyle= 'dashed', linewidths=0.2, cmap=cmap)
    ax.set_yticks(np.arange(AUC.shape[0]) + 0.5, minor=False)
    ax.set_xticks(np.arange(AUC.shape[1]) + 0.5, minor=False)
    ax.set_xticklabels(xticklabels, minor=False)
    ax.set_yticklabels(yticklabels, minor=False)
    plt.title(title)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)      

    # Remove last blank column
    plt.xlim( (0, AUC.shape[1]) )

    # Turn off all the ticks
    ax = plt.gca()    
    for t in ax.xaxis.get_major_ticks():
        t.tick1On = False
        t.tick2On = False
    for t in ax.yaxis.get_major_ticks():
        t.tick1On = False
        t.tick2On = False

    # Add color bar
    plt.colorbar(c)

    # Add text in each cell 
    show_values(c)

    # Proper orientation (origin at the top left instead of bottom left)
    if correct_orientation:
        ax.invert_yaxis()
        ax.xaxis.tick_top()       

    # resize 
    fig = plt.gcf()
    fig.set_size_inches(cm2inch(figure_width, figure_height))

In [ ]:
def plot_classification_report(classification_report, save=False, filename=False, cmap='RdBu'):
    lines = classification_report.split('\n')
    class_names = list(class_dict.keys())
    plotMat = []
    support = []
    #class_names = []
    #count = 0
    for line in lines[2 : (len(lines) - 5)]:
        t = line.strip().split()
        if len(t) < 2: continue
        v = [float(x) for x in t[1: len(t) - 1]]
        support.append(int(t[-1]))
        plotMat.append(v)

    xlabel = 'Metrics'
    ylabel = 'Classes'
    xticklabels = ['Precision', 'Recall', 'F1-score']
    ytick_labels = [f'{class_names[i]}({sup})' for i, sup in enumerate(support) ]
    
    #print(len(support))
    yticklabels = ['{0} ({1})'.format(class_names[idx], sup) for idx, sup  in enumerate(support)]
    figure_width = 25
    figure_height = len(class_names) + 7
    correct_orientation = False
    heatmap(np.array(plotMat), 'classification report', xlabel, ylabel, xticklabels, yticklabels, figure_width, figure_height, correct_orientation, cmap=cmap)
    #plt.tight_layout()
    if save:
        plt.savefig(filename, bbox_inches = 'tight')
    plt.show()
    plt.close()

In [ ]:
cr = classification_report(y_pred_df['y_actual'], y_pred_df['y_pred'])
plot_classification_report(cr)